In [7]:
using Revise

using AutomotiveDrivingModels
using AutoViz
using AutomotiveSensors
using AutomotivePOMDPs
using Parameters
using StaticArrays
using ProfileView

using GridInterpolations 
using POMDPs
using POMDPToolbox
using QMDP
using JLD
using Reel

using ProfileView



include("../src/pomdp_types.jl")
include("../src/spaces.jl")
include("../src/transition.jl")
include("../src/observation.jl")
include("../src/belief.jl")
include("../src/policy.jl")
include("../src/frenet_pedestrian_pomdp.jl")
include("../src/helpers.jl")
include("../src/rendering.jl")

#include("../src/PedestrianAvoidancePOMDP.jl")

#using PedestrianAvoidancePOMDP

In [8]:
policy = load("../policy/policy.jld")["policy"];

In [9]:
include("../src/PedestrianAvoidancePOMDP.jl")

pomdp = SingleOCFPOMDP()
updater = SingleOCFUpdater(pomdp)

params = CrosswalkParams()
params.roadway_length = 400.0
params.obstacles_visible = false

obstacle_offset = -2
obstacle_1 = ConvexPolygon([VecE2(34, obstacle_offset), VecE2(34, obstacle_offset-3), VecE2(46.5, obstacle_offset-3), VecE2(46.5, obstacle_offset)],4)
obstacle_2 = ConvexPolygon([VecE2(34, +4.5), VecE2(34, +7.5), VecE2(46.5, +7.5), VecE2(46.5, +4.5)],4)
params.obstacles = [obstacle_1, obstacle_2]
#params.obstacles = [obstacle_2]

env = CrosswalkEnv(params)
pomdp.env = env


ego_id = 1
ped_id = 2
ped2_id = 3


ego_v = 11.11
ped_v = 5.4/3.6
hitpoint = 50             # 50: middle, 100: left corner, 0: right corner
ped_theta = π/2


# fix values
ped_x = 50.0
ped_y_start = -5.0
ego_y = 0.0


ped_y_offset_hitpoint = VehicleDef().width * hitpoint / 100 - VehicleDef().width/2
ped_y_end = ped_y_offset_hitpoint
ped_t_collision = (ped_y_end - ped_y_start) / ped_v; 
ego_x = ped_x - ego_v * ped_t_collision - VehicleDef().length/2;

if ( false )
    ego_v = 0.0
    ego_x = 20
    ped_x = 30
    ped_y_start = -2.0
    ped_v = 0.0
end

ped_y_start = -5.0
ped_v = 2.0

# Car definition
ego_initial_state = VehicleState(VecSE2(ego_x, ego_y, 0.), env.roadway.segments[1].lanes[1], env.roadway, ego_v)
ego = Vehicle(ego_initial_state, VehicleDef(), 1)

# Pedestrian definition 
ped_initial_state = VehicleState(VecSE2(ped_x,ped_y_start,ped_theta), env.crosswalk, env.roadway, ped_v)
ped = Vehicle(ped_initial_state, AutomotivePOMDPs.PEDESTRIAN_DEF, 2)

ped2_v = 0.0
ped2 = Vehicle(VehicleState(VecSE2(54., 5., 0.78), env.crosswalk, env.roadway, ped2_v), AutomotivePOMDPs.PEDESTRIAN_DEF, ped2_id)


scene = Scene()
push!(scene, ego)
push!(scene, ped)
push!(scene, ped2)



cam = FitToContentCamera(0.);
timestep = 0.2


pos_noise = 0.001
vel_noise = 0.000
false_positive_rate = 0.00
false_negative_rate = 0.00
rng = MersenneTwister(1);
sensor = AutomotiveSensors.GaussianSensor(AutomotiveSensors.LinearNoise(10, pos_noise, 0.00), 
                 AutomotiveSensors.LinearNoise(10, vel_noise, 0.00), false_positive_rate, false_negative_rate, rng) 



# define a model for each entities present in the scene
models = Dict{Int, DriverModel}()

models[ego_id] = FrenetPedestrianPOMDP(a=LatLonAccel(0.0, 0.0),env=env,sensor=sensor, obstacles=env.obstacles, timestep=timestep,
    env=env,
    pomdp=pomdp,
    policy=policy,
    updater=updater,
    ego_vehicle=ego,
    desired_velocity=ego_v,
    b=initBeliefAbsentPedestrian(pomdp, ego_y, ego_v)
)
models[ego_id].pomdp.desired_velocity = ego_v
models[ego_id].pomdp.ΔT = timestep

models[ped_id] = ConstantPedestrian(v_desired=ped.state.v, dawdling_amp=0.001) # dumb model
models[ped2_id] = ConstantPedestrian(v_desired=ped2.state.v, dawdling_amp=0.001) # dumb model


nticks = 50
rec = SceneRecord(nticks+1, timestep)

risk = Float64[]
sensor_observations = [Vehicle[]]
belief = Dict{Int64, SingleOCFBelief}[]
ego_vehicle = Vehicle[]
action_pomdp = SingleOCFAction[]

obs_callback = (ObservationCallback(risk,sensor_observations, belief, ego_vehicle, action_pomdp),)

tic()
simulate!(rec, scene, env.roadway, models, nticks, obs_callback)
toc()




--------------------------POMDP high level planner----------------------- t: 0.0
EGO: x/y:10.966666666666669 / 0.0 v: 11.11
Vehicle(2, VehicleState(VecSE2({50.000, -5.001}, 1.571), Frenet(RoadIndex({1, 0.125001}, {1, 1}), 50.000, -5.001, 1.571), 2.000), VehicleDef(PEDESTRIAN, 1.000, 1.000))
Vehicle(3, VehicleState(VecSE2({54.000, 5.002}, 0.780), Frenet(RoadIndex({1, 0.135001}, {1, 2}), 54.000, 2.002, 0.780), 0.000), VehicleDef(PEDESTRIAN, 1.000, 1.000))
action combined: 

[1.0, 1.0]
--------------------------POMDP high level planner----------------------- t: 0.2
EGO: x/y:13.208666666666668 / 0.2 v: 11.309999999999999
Vehicle(2, VehicleState(VecSE2({50.000, -4.599}, 1.571), Frenet(RoadIndex({1, 0.125001}, {1, 1}), 50.000, -4.599, 1.571), 2.000), VehicleDef(PEDESTRIAN, 1.000, 1.000))
Vehicle(3, VehicleState(VecSE2({53.999, 5.001}, 0.780), Frenet(RoadIndex({1, 0.134999}, {1, 2}), 53.999, 2.001, 0.780), 0.000), VehicleDef(PEDESTRIAN, 1.000, 1.000))
b-length: 20
action combined: [1.0, 1.0]
--------------------------POMDP high level planner----------------------- t: 0.4
EGO: x/y:15.490666666666668 / 0.4 v: 11.509999999999998
Vehicle(2, VehicleState(VecSE2({49.998, -4.200}, 1.571), Frenet(RoadIndex({1, 0.124996}, {1, 1}), 49.998, -4.200, 1.571), 2.000), VehicleDef(PEDESTRIAN, 1.000, 1.000))
Vehicle(3, VehicleState(VecSE2({54.001, 5.001}, 0.780), Frenet(RoadIndex({1, 0.135002}, {1, 2}), 54.001, 2.001, 0.780), 0.000), VehicleDef(PEDESTRIAN, 1.000, 1.000))
  0.06

b-length: 112
  0.508229 seconds (1.11 M allocations: 177.916 MiB, 49.21% gc time)
b-length: 99
  0.207285 seconds (862.82 k allocations: 117.138 MiB, 13.66% gc time)
action combined: [0.0, 0.0]
--------------------------POMDP high level planner----------------------- t: 2.8000000000000003
EGO: x/y:45.67466666666667 / 1.0 v: 13.109999999999992
Vehicle(2, VehicleState(VecSE2({50.001, 0.401}, 1.571), Frenet(RoadIndex({1, 0.125003}, {1, 1}), 50.001, 0.401, 1.571), 1.500), VehicleDef(PEDESTRIAN, 1.000, 1.000))
Vehicle(3, VehicleState(VecSE2({54.000, 4.716}, 0.780), Frenet(RoadIndex({1, 0.135001}, {1, 2}), 54.000, 1.716, 0.780), -0.500), VehicleDef(PEDESTRIAN, 1.000, 1.000))
b-length: 94
  0.232739 seconds (938.12 k allocations: 135.585 MiB, 13.91% gc time)
b-length: 96
  0.212954 seconds (849.47 k allocations: 115.779 MiB, 14.96% gc time)
action combined: [0.0, 0.0]
-----------------> Collision <----------------------
elapsed time: 5.700196912 seconds


5.700196912

In [10]:
# Visualize scenario
duration, fps, render_hist = animate_record(rec, timestep, env, sensor, sensor_observations, risk, belief, ego_vehicle, action_pomdp, CarFollowCamera(2,10.0))
film = roll(render_hist, fps = fps, duration = duration)

Reel.Frames{MIME{Symbol("image/png")}}("/tmp/tmprBBya1", 0x0000000000000010, 5.0, nothing)